 # Merging Datasets, Cleaning and Filtering Dataframe, & Diet Function -> Deliverable A: Data on Prices for Different Food

In [2]:
# Importing packages
import pandas as pd
import re 

In [26]:
# Creating and cleaning data
def drop_nan_columns(df):
    # Drop columns where all values are NaN
    df_cleaned = df.dropna(axis=1, how='all')
    return df_cleaned

# USDA Data for Prices
usda_data = pd.read_csv("USDA-305tj(Sheet1).csv")
# Trader Joes Data for Ingredients
tj_data = pd.read_csv('trader_joes.csv')
tj_data.rename(columns={"gtin_upc":"GTIN/UPC"}, inplace=True)

# Merging data
merged_data = pd.merge(usda_data, tj_data, how = 'outer', on = 'GTIN/UPC' )
merged_data = drop_nan_columns(merged_data)

# Data of complete pricing
completed_prices = pd.read_csv("EEP153_COMPLETED_PRICES.csv")
completed_prices['GTIN/UPC'] = completed_prices['GTIN/UPC'].astype(int)

# Dropping unneeded columns and duplicates
not_needed = ['branded_food_category', 'data_source', 'modified_date',
            'available_date', 'market_country', 'Unnamed: 0', 'brand_owner', 'Market Country',
            'brand_name', 'Brand Owner', 'Brand', 'fdc_id']

merged_data_drop = merged_data.drop(not_needed, axis = 1)
merged_data_drop.drop_duplicates()

# Setting index, and cleaning data
merged_data_drop = merged_data_drop.set_index('Name')
merged_data_clean =  merged_data_drop[merged_data_drop.index.notna()]
merged_data_clean = merged_data_clean.reset_index().drop(['Price', 'Name'], axis = 1)
merged_data_clean = merged_data_clean[~merged_data_clean['GTIN/UPC'].duplicated(keep='first')]

# Final data frame, before cleaning
final_data = pd.merge(completed_prices, merged_data_clean, on = 'GTIN/UPC', how = 'left')

# Cleaning Data Frame
final_data['ingredients'] = final_data.apply(
    lambda row: f"{row['ingredients_x']}, {row['ingredients_y']}" if pd.notna(row['ingredients_x']) and pd.notna(row['ingredients_y']) 
    else row['ingredients_x'] if pd.notna(row['ingredients_x']) 
    else row['ingredients_y'], axis=1
)

# Filling in missing ingredients
fill_ingredients = ['WHEAT FLOUR, SUGAR, SALT, BARLEY MALT SYRUP',\
                     'POPCORN, SUNFLOWER OIL, SUGAR, SALT', 'SOLID WHITE TUNA, WATER, SALT']

final_data.loc[final_data['ingredients'].isna(), 'ingredients'] = fill_ingredients
final_data = final_data.drop(['ingredients_x', 'ingredients_y'], axis = 1).set_index('Name')

# Final dataset
final_data

# Converting to CSV
final_data.to_csv('final_data_tj.csv', index=False)

In [27]:
# Function to filter out dataframe depending on certain strings
def remove_rows(df, column_name, search_strings):
    # Convert all values in the specified column to strings
    df[column_name] = df[column_name].apply(lambda x: str(x) if x is not None else "")
    # Create a regex pattern to match any of the search strings as whole words
    search_pattern = r'|'.join([r'\b' + re.escape(search_string) + r'\b' for search_string in search_strings])
    # Filtering out rows where the column contains any of the search strings
    df_filtered = df[~df[column_name].str.contains(search_pattern, case=False, na=False, regex=True)]
    return df_filtered


# Function to keep certain values depending on certain strings
def keep_rows(df, column_name, search_strings):
    # Converts all values to strings
    df[column_name] = df[column_name].apply(lambda x: str(x) if x is not None else "")
    # Create a regex pattern for exact matches
    search_pattern = r'|'.join([r'\b' + re.escape(search_string) + r'\b' for search_string in search_strings])
    # Filter rows where the column matches any of the exact strings in search_strings
    filtered_df = df[df[column_name].str.contains(search_pattern, case=False, na=False)]
    return filtered_df



In [ ]:
# Vegetarian Dataframe
meats_and_fish = ['CHICKEN', 'BEEF', 'HAM', 'PORK', 'FISH', 'TURKEY', 'SALMON', 'TUNA',
                  'SHRIMP', 'LOBSTER', 'CRAB', 'SCALLOPS', 'CLAMS', 'OYSTERS', 'GELATINE']
veg_df = remove_rows(final_data, 'ingredients', meats_and_fish)
# Converting to CSV
veg_df.to_csv('vegetarian_tj.csv', index=False)

In [ ]:
# Vegan Dataframe
animal_products = ['CHICKEN', 'BEEF', 'HAM', 'PORK', 'FISH','TURKEY', 'SALMON', 'TUNA',
                    'MILK', 'BUTTER', 'EGG', 'EGGS', 'HONEY', 'CHEESE', 'YOGURT', 'CREAM', 'GELATINE',
                    'SHRIMP', 'LOBSTER', 'CRAB', 'SCALLOPS', 'CLAMS', 'OYSTERS']

vegan_df = remove_rows(final_data, 'ingredients', animal_products)
# Converting to CSV
vegan_df.to_csv('vegan_tj.csv', index=False)

In [ ]:
# Carnivore Dataframe
carnivore_df = keep_rows(final_data, 'ingredients', animal_products)
# Converting to CSV
carnivore_df.to_csv('carnivore_tj.csv', index=False)

In [24]:
# Allergen Free Dataframe
allergen_free = ['MILK', 'BUTTER', 'CHEESE', 'YOGURT', 'CREAM', 
                     'EGG', 'EGGS', 
                     'WHEAT', 'BARLEY', 'RYE', 'MALT', 'OATS', 
                     'PEANUTS', 'ALMONDS', 'CASHEWS', 'WALNUTS', 'PISTACHIOS', 'PECANS', 'HAZELNUTS', 
                     'SOY', 'SOYBEAN', 'SOY LECITHIN', 'TOFU', 
                     'SHRIMP', 'LOBSTER', 'CRAB', 'SCALLOPS', 'CLAMS', 'OYSTERS']

allergen_free_df = remove_rows(final_data, 'ingredients', allergen_free)

# Converting to CSV
allergen_free_df.to_csv('allergen_free_tj.csv', index=False)

In [25]:
# High Fat and Low Carb Diet DF

hflc = ['CHICKEN', 'BEEF', 'HAM', 'PORK', 'FISH','TURKEY', 'SALMON', 'TUNA',
                    'MILK', 'BUTTER', 'EGG', 'EGGS', 'HONEY', 'CHEESE', 'YOGURT', 'CREAM', 'GELATINE',
                    'SHRIMP', 'LOBSTER', 'CRAB', 'SCALLOPS', 'CLAMS', 'OYSTERS', 'OIL', 'BERRIES',
                    'AVOCADO', 'PEANUTS', 'ALMONDS', 'CASHEWS', 'WALNUTS', 'PISTACHIOS', 'PECANS', 'HAZELNUTS']

hflc_df = keep_rows(final_data, 'ingredients', hflc)
# Converting to CSV
hflc_df.to_csv('hflc_tj.csv', index=False)